In [8]:
'''
This Notebook demonstrates UAA and Asset cloudFoundary services.
A simple web-based application is used to demonstrate their use. 

The python-based script uses:
    cloudFoundry cli, api, curl
    python subprocess (standard library)
    json (python standard library)
    UAA and GE Asset micrososervices

References:
Based on the example at
    https://www.predix.io/blog/article.html?article_id=2034
with additional information from 
    http://stackoverflow.com/questions/27985469/how-to-get-oauth-token-from-cloudfoundry
    https://docs.python.org/3.7/reference/index.html
    https://docs.python.org/3.7/library/index.html
'''

x=1

In [9]:
'''
Procedure:

Login
1> mkdir ~/tempForBlog1; cd ~/tempForBlog1
2> cf login

Create UAA Service
3> cf create-service predix-uaa Tiered my-uaa -c '{"adminClientSecret":"my-secret"}'

Get UAA Vitals
4> appname=`whoami`.temp.html # need unique name
5> touch $appname
6> cf push --no-start $appname
7> cf bs $appname my-uaa 
8> cf env $appname # Output is environment variables
9> cf delete $appname --f

Create Asset Service
10> cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

Get Asset Vitals
11> touch $appname
12> cf push --no-start $appname
13> cf bs $appname my-asset
14> cf env $appname

UAAC Targets UAA
15> uaac target https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io
16> uaac token client get admin --secret my-secret # gets the admin token, NOT usable for asset

Add UAAC Client
17> uaac client add asset-client --authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --autoapprove openid --authorized_grant_types authorization_code,client_credentials,refresh_token,password --secret my-secret --name asset-client

Specific commands:

As outlined above a dummy app is created as means to get the "issuerId".  In order to script this it
is necessary to reproducing the process using cf curl.  This involves tracing JSON urls to find it.
The final target is:
cf env cwinsor.temp.html ...
 "issuerId": "https://5b5428fa-4922-4be6-81ed-ad40a2a28f70.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token",

To reproduce:
cf curl /v2/spaces
produces
["resources"][0]["metadata"]["url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc",
["resources"][0]["entity"]["name"] = "dev"
["resources"][0]["entity"]["apps_url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps",
["resources"][0]["entity"]["service_instances_url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances",

<first find the UAA service GUID>
cf curl /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances
produces
["resources"][0]["entity"]["name"]: "my-uaa"
["resources"][0]["metadata"]["guid"]: "5b5428fa-4922-4be6-81ed-ad40a2a28f70",

<now that we have the UAA GUID - search for that in my app as a service...>
cf curl /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps
produces
["resources"][0]["entity"]["name"]: "cwinsor.temp.html"
["resources"][0]["metadata"]["url"]: "/v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d",
["resources"][0]["entity"]["service_bindings_url"]: "/v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d/service_bindings",

cf curl /v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d/service_bindings
produces
["resources"][0]["entity"][service_instance_guid": "5b5428fa-4922-4be6-81ed-ad40a2a28f70",
["resources"][0]["entity"]["credentials"]["issuerId"]: "https://5b5428fa-4922-4be6-81ed-ad40a2a28f70.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token",

'''
x = 1

In [10]:
### Log In

import getpass
import subprocess
import json

print("Log in")
finished = False
while not finished:
    subprocess.call(["cf", "logout"])
    u = raw_input("username:")
    p = getpass.getpass("password:")
    proc = subprocess.Popen(["cf", "login", "-s", "dev", "-u", u, "-p", p],
                        shell=False,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
    stdout_value, stderr_value = proc.communicate()
    #print '\tstdout: ', repr(stdout_value)
    #print '\tstderr: ', repr(stderr_value)
    #print '\tproc.returncode: ', proc.returncode
    if proc.returncode == 0:
        finished = True
    else:
        print "login failed"

# squirrel away URLs for the space, apps and services
temp_p = json.loads(check_output(["cf", "curl", "/v2/spaces"]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "dev"):
        MY_DEV_SPACE_URL = line["metadata"]["url"]
        MY_APPS_URL      = line["entity"]["apps_url"]
        MY_SERVICES_URL  = line["entity"]["service_instances_url"]
        
print ("MY_DEV_SPACE_URL: " + MY_DEV_SPACE_URL)
print ("MY_APPS_URL: " +  MY_APPS_URL)
print ("MY_SERVICES_URL: " +  MY_SERVICES_URL)
print "done"

Log in
username:cwinsor@gmail.com
password:········
MY_DEV_SPACE_URL: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc
MY_APPS_URL: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps
MY_SERVICES_URL: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances
done


In [11]:
# Create UAA Service
# 3> cf create-service predix-uaa Tiered my-uaa -c '{"adminClientSecret":"my-secret"}'

from subprocess import check_output

print "Creating UAA Service..."
secret = getpass.getpass('secret code:')
aa = "\"adminClientSecret\""
bb = "\"" + secret + "\""
cc = "{" + aa + ":" + bb + "}"
#print cc
dd = ["cf", "create-service", "predix-uaa", "Free", "my-uaa", "-c", cc]
#print dd
foo = check_output(dd)
print foo

print "done"

Creating UAA Service...
secret code:········
Creating service instance my-uaa in org cwinsor@gmail.com / space dev as cwinsor@gmail.com...
OK

done


In [14]:
# Get uaa vitals
# We need to get the environment variables of our new UAA.
# One way to accomplish that is to bind to a dummy application.
# Here we create a simple empty html file to serve as our dummy app.

import subprocess
import json

print "Get UAA vitals (will take a few seconds...)"
# create a temporary application
subprocess.check_call(["echo", "", ">", "cwinsor.temp.html"])
subprocess.check_call(["cf", "push", "--no-start", "cwinsor.temp.html"])
subprocess.check_call(["cf", "bs", "cwinsor.temp.html", "my-uaa"])
subprocess.check_call(["cf", "env", "cwinsor.temp.html"])

# we now will use that application to get environment variables relating to UAA

# find the UAA service GUID
# starting from the services URL, and search for my-uaa
# then from that URL get the GUID
temp_p = json.loads(check_output(["cf", "curl", MY_SERVICES_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "my-uaa"):
        MY_UAA_SERVICE_URL = line["metadata"]["url"]
print ("MY_UAA_SERVICE_URL:" + MY_UAA_SERVICE_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_UAA_SERVICE_URL]))
MY_UAA_SERVICE_GUID = temp_p["metadata"]["guid"]
print ("MY_UAA_SERVICE_GUID: " + MY_UAA_SERVICE_GUID)

# find the app
temp_p = json.loads(check_output(["cf", "curl", MY_APPS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_1_URL = line["metadata"]["url"]
#print ("MY_APP_1_URL:" + MY_APP_1_URL)

# find the issuerId of the service instance matching the UAA service
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))
MY_APP_1_SERVICE_BINDINGS_URL = temp_p["entity"]["service_bindings_url"]
#print ("MY_APP_1_SERVICE_BINDINGS_URL: " + MY_APP_1_SERVICE_BINDINGS_URL)

temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_SERVICE_BINDINGS_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["service_instance_guid"] == MY_UAA_SERVICE_GUID):
        MY_UAA_CREDENTIALS_ISSUER_ID = line["entity"]["credentials"]["issuerId"]
print ("MY_UAA_CREDENTIALS_ISSUER_ID: " + MY_UAA_CREDENTIALS_ISSUER_ID)

# delete the app
subprocess.check_call(["cf", "delete", "cwinsor.temp.html", "-f"])

print "done"

Get UAA vitals (will take a few seconds...)
MY_UAA_SERVICE_URL:/v2/service_instances/e3728699-10f8-4c2d-b496-d794267e0ecf
MY_UAA_SERVICE_GUID: e3728699-10f8-4c2d-b496-d794267e0ecf
MY_UAA_CREDENTIALS_ISSUER_ID: https://e3728699-10f8-4c2d-b496-d794267e0ecf.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token
done


In [15]:
# Create Asset Service
# 10> cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

from subprocess import check_output

print "Creating Asset Service..."
issuer_id = json.dumps({"trustedIssuerIds": [MY_UAA_CREDENTIALS_ISSUER_ID]})
foo = check_output(["cf", "create-service", "predix-asset", "Tiered", "my-asset", "-c", issuer_id])

print "done"

Creating Asset Service...
done


In [16]:
# Get Asset vitals
# We need to get the environment variables of our Asset.
# One way to accomplish that is to bind to a dummy application.
# Here we create a simple empty html file to serve as our dummy app.

import subprocess
import json

print "Get Asset vitals (will take a few seconds...)"
# create a temporary application
subprocess.check_call(["echo", "", ">", "cwinsor.temp.html"])
subprocess.check_call(["cf", "push", "--no-start", "cwinsor.temp.html"])
subprocess.check_call(["cf", "bs", "cwinsor.temp.html", "my-asset"])
subprocess.check_call(["cf", "env", "cwinsor.temp.html"])

# squirrel away the URL for the Asset Service
temp_p = json.loads(check_output(["cf", "curl", MY_SERVICES_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "my-asset"):
        MY_ASSET_SERVICE_URL = line["metadata"]["url"]
print ("MY_ASSET_SERVICE_URL: " + MY_ASSET_SERVICE_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_ASSET_SERVICE_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

MY_ASSET_SERVICE_GUID = temp_p["metadata"]["guid"]
print ("MY_ASSET_SERVICE_GUID: " + MY_ASSET_SERVICE_GUID)


# find the app
temp_p = json.loads(check_output(["cf", "curl", MY_APPS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_1_URL = line["metadata"]["url"]
print ("MY_APP_1_URL:" + MY_APP_1_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

MY_APP_1_SERVICE_BINDINGS_URL = temp_p["entity"]["service_bindings_url"]
print ("MY_APP_1_SERVICE_BINDINGS_URL: " + MY_APP_1_SERVICE_BINDINGS_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_SERVICE_BINDINGS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["credentials"]["instanceId"] == MY_ASSET_SERVICE_GUID):
        MY_ASSET_SERVICE_URI = line["entity"]["credentials"]["uri"]
print ("MY_ASSET_SERVICE_URI: " + MY_ASSET_SERVICE_URI)
print "done"

Get Asset vitals (will take a few seconds...)
MY_ASSET_SERVICE_URL: /v2/service_instances/d718b52b-93ed-493f-b128-d77648fd7d4a
MY_ASSET_SERVICE_GUID: d718b52b-93ed-493f-b128-d77648fd7d4a
MY_APP_1_URL:/v2/apps/1071557c-409f-4434-822b-adf30b0eda4f
MY_APP_1_SERVICE_BINDINGS_URL: /v2/apps/1071557c-409f-4434-822b-adf30b0eda4f/service_bindings
MY_ASSET_SERVICE_URI: https://predix-asset.run.aws-usw02-pr.ice.predix.io
done


In [17]:
# UAAC Targets UAA

import subprocess
import json
import getpass

print "UAAC Targets UAA"
uaa_credentials_issuer = MY_UAA_CREDENTIALS_ISSUER_ID.split("/oauth/token")
subprocess.check_call(["uaac", "target", uaa_credentials_issuer[0]])

my_secret = getpass.getpass("UAAC secret:")
subprocess.check_call(["uaac", "token", "client", "get", "admin", "--secret", my_secret])

print "done"
    

UAAC Targets UAA
UAAC secret:········
done


In [18]:
# Add UAAC Client
# We need to use the instance id of Asset, sometimes called the Predix zone id

# 17> uaac client add asset-client --authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --autoapprove openid --authorized_grant_types authorization_code,client_credentials,refresh_token,password --secret my-secret --name asset-client

import subprocess
import json
import getpass

print "Add UAAC Client"
my_secret = getpass.getpass("UAAC secret:")
instanceid = "predix-asset.zones." + MY_ASSET_SERVICE_GUID + ".user"
l = ["uaac",
     "client",
     "add",
     "asset-client",
     "--authorities",
     "openid,uaa.none,uaa.resource," + instanceid,
     "--scope",
     "uaa.none,openid," + instanceid,
     "--autoapprove",
     "openid",
     "--authorized_grant_types",
     "authorization_code,client_credentials,refresh_token,password",
     "--secret",
     my_secret,
     "--name",
     "asset-client"]

subprocess.check_call(l)
print "done"

Add UAAC Client
UAAC secret:········
done


In [19]:
# Get Then Display The Token
import subprocess

#18> uaac token client get asset-client --secret my-secret # if token expires then do this again
#19> uaac context

print "Get Then Display The Token"
my_secret = getpass.getpass("UAAC secret:")

subprocess.check_call(["uaac", "token", "client", "get", "asset-client", "--secret", my_secret])
foo = check_output(["uaac", "context"])
#print(json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': ')))
#print foo
bar1 = foo.split("access_token: ")
bar2 = bar1[1].split("\n")
MY_BEARER_ACCESS_TOKEN = bar2[0]
print ("MY_BEARER_ACCESS_TOKEN: " + MY_BEARER_ACCESS_TOKEN)
print "done"


Get Then Display The Token
UAAC secret:········
MY_BEARER_ACCESS_TOKEN: eyJhbGciOiJSUzI1NiIsImtpZCI6ImxlZ2FjeS10b2tlbi1rZXkiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiJhNTc3M2JmNDhhYWE0NWE2YjMxMGQzYzM3NDEyNGZiOSIsInN1YiI6ImFzc2V0LWNsaWVudCIsInNjb3BlIjpbInByZWRpeC1hc3NldC56b25lcy5kNzE4YjUyYi05M2VkLTQ5M2YtYjEyOC1kNzc2NDhmZDdkNGEudXNlciIsInVhYS5yZXNvdXJjZSIsIm9wZW5pZCIsInVhYS5ub25lIl0sImNsaWVudF9pZCI6ImFzc2V0LWNsaWVudCIsImNpZCI6ImFzc2V0LWNsaWVudCIsImF6cCI6ImFzc2V0LWNsaWVudCIsImdyYW50X3R5cGUiOiJjbGllbnRfY3JlZGVudGlhbHMiLCJyZXZfc2lnIjoiZTY0OTE4ZDYiLCJpYXQiOjE0ODExNTI2MzcsImV4cCI6MTQ4MTE5NTgzNywiaXNzIjoiaHR0cHM6Ly9lMzcyODY5OS0xMGY4LTRjMmQtYjQ5Ni1kNzk0MjY3ZTBlY2YucHJlZGl4LXVhYS5ydW4uYXdzLXVzdzAyLXByLmljZS5wcmVkaXguaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiJlMzcyODY5OS0xMGY4LTRjMmQtYjQ5Ni1kNzk0MjY3ZTBlY2YiLCJhdWQiOlsidWFhIiwiYXNzZXQtY2xpZW50Iiwib3BlbmlkIiwicHJlZGl4LWFzc2V0LnpvbmVzLmQ3MThiNTJiLTkzZWQtNDkzZi1iMTI4LWQ3NzY0OGZkN2Q0YSJdfQ.QLrhhADchg4UcX1rg9tPTafR75ebXIgyJKyalY6txqZn1rhWrjtOnGAYg9AjxXtTxH_0-0sYpRymwJxr44POZu

In [20]:
# Validation By Talking To Asset
print "Validation By Talking To Asset"

#foo3 = ["curl",
#        "-H", [predix-zone-id: " + MY_ASSET_SERVICE_GUID + """,
#       "-H", "\"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\"",
#        MY_ASSET_SERVICE_URI]
#bar = check_output(foo3)
#print bar

#print "command 1"
THE_CMD  = " curl "
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI

proc = subprocess.Popen(THE_CMD + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)


#curl -X POST -d '[{ "uri": "/tests/my-first-test", "foo": true, "bar": [34,56] }]'
#curl -X POST -d '[{ "uri": "/tests/my-first-test", "foo": true, "bar": [34,56] }]' 
#-H "Content-Type: application/json"
#-H "predix-zone-id: 87af97a0-c4a9-49fd-a002-240aacb3c540"
#-H "Authorization: <BEARER-TOKEN>"
#https://predix-asset.run.aws-usw02-pr.ice.predix.io/tests

#print "command 2"
THE_CMD  = ' curl -X POST -d \'[{ "uri": "/tests/my-first-test", "foo": true, "bar": [34,56] }]\' '
CONT_HDR = " -H \"Content-Type: application/json\" "
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI + "/tests "

proc = subprocess.Popen(THE_CMD + CONT_HDR + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)


# curl -H "predix-zone-id: 87af97a0-c4a9-49fd-a002-240aacb3c540" -H "Authorization: <BEARER-TOKEN>" https://predix-asset.run.aws-usw02-pr.ice.predix.io/tests/my-first-test
#print "command 3"
THE_CMD  = ' curl '
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI + "/tests/my-first-test "

proc = subprocess.Popen(THE_CMD + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)


# curl -H "predix-zone-id: 5aacd7de-f6da-42f6-8e3f-6c6c6dd595fd" -H "Authorization: <BEARER-TOKEN>" https://predix-asset.run.aws-usw02-pr.ice.predix.io 

#print "command 4"
THE_CMD  = ' curl '
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI

proc = subprocess.Popen(THE_CMD + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)

print "done"

Validation By Talking To Asset
	stdout: '[]'
	stdout: ''
	stdout: '[\n  {\n    "uri": "/tests/my-first-test",\n    "foo": true,\n    "bar": [\n      34,\n      56\n    ]\n  }\n]'
	stdout: '[{"collection":"tests","count":1}]'
done


In [34]:

#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
